# Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Load
#### 아파트 실거래가 (출처 : 국토교통부)
#### 계약일 기준 / 전용면적 / 금액 만원 단위
#### 40이하(극소형) / 40~60(소형) / 60~85(중소형) / 85~102(중형) / 102~135(중대형) / 135(대형)

In [22]:
apt_1801 = pd.read_csv('Apt sales real price_1801.csv', encoding='CP949')
apt_1802 = pd.read_csv('Apt sales real price_1802.csv', encoding='CP949')
apt_1803 = pd.read_csv('Apt sales real price_1803.csv', encoding='CP949')
apt_1804 = pd.read_csv('Apt sales real price_1804.csv', encoding='CP949')
apt_1805 = pd.read_csv('Apt sales real price_1805.csv', encoding='CP949')
apt_1806 = pd.read_csv('Apt sales real price_1806.csv', encoding='CP949')
apt_1807 = pd.read_csv('Apt sales real price_1807.csv', encoding='CP949')
apt_1808 = pd.read_csv('Apt sales real price_1808.csv', encoding='CP949')
apt_1809 = pd.read_csv('Apt sales real price_1809.csv', encoding='CP949')
apt_1810 = pd.read_csv('Apt sales real price_1810.csv', encoding='CP949')
apt_1811 = pd.read_csv('Apt sales real price_1811.csv', encoding='CP949')
apt_1812 = pd.read_csv('Apt sales real price_1812.csv', encoding='CP949')

In [23]:
apt = pd.concat([apt_1801,apt_1802,apt_1803,apt_1804,apt_1805,apt_1806,apt_1807,apt_1808,apt_1809,apt_1810,apt_1811,apt_1812])

In [24]:
apt.shape

(501308, 12)

In [25]:
apt[['region_1', 'region_2', 'region_3']] = apt['si_gun_gu'].str.split(" ", n=2, expand=True)

In [26]:
def func1(x):
    if 0 < x <= 40:
        return '극소형'
    elif 40 < x <= 60:
        return '소형'
    elif 60 < x <= 85:
        return '중소형'
    elif 85 < x <= 102:
        return '중형'
    elif 102 < x <= 135:
        return '중대형'
    return '대형'

apt['area_cate'] = apt['dedicated_area'].apply(func1)

In [27]:
apt['cost_of_unit'] = apt['amt_10000'] / (apt['dedicated_area'] / 3.3)

# 현황

## 1. 시도별 아파트 매매 현황 (신고된 계약일자 기준)

In [28]:
pd.pivot_table(apt, index='region_1', columns='contract_yymm', values='amt_10000', aggfunc='count')

contract_yymm,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812
region_1,,,,,,,,,,,,
강원도,1081,949,1401,1065,1004,1458,923,1043,1015,1184,857,771
경기도,13866,12695,15752,9758,10044,10379,10754,19680,20063,13549,8171,5808
경상남도,2471,1930,2681,2053,2231,1762,1649,1752,1938,2656,2298,1548
경상북도,1726,1515,2072,1535,1525,1620,1355,1489,1442,1833,1387,1220
광주광역시,2090,1860,2747,1936,2058,2007,1987,2480,2619,2511,1708,1251
대구광역시,2846,2848,3830,2501,2605,3406,1976,2715,3051,2986,2197,1290
대전광역시,1660,1476,1968,1261,1314,1267,1150,1614,2103,2621,1768,1284
부산광역시,2740,2372,3126,2045,2195,1862,1610,1835,2033,2325,1835,1353
서울특별시,12563,9162,9401,4485,4697,5236,7031,14971,7210,3269,1776,1121


## 2. 시도별 가장 많이 상승한 아파트 상위 10개 단지

In [131]:
# 아파트 단지별로 월 평단가 평균계산
apt_mm_unit_amt=apt.groupby(['si_gun_gu','region_1','region_2','region_3','apt_name','contract_yymm'])['cost_of_unit'].mean().reset_index()

# 월 평균 증가율 함수
def fun_cagr(values):
    values = list(values)
    n = len(values)
    if n == 1:
        res = 0
    else:
        res = (values[n-1]/values[0]) ** (1/(n-1)) - 1
    return res

In [137]:
ex2_1 = apt_mm_unit_amt.groupby(['si_gun_gu','region_1','region_2','region_3','apt_name']).cost_of_unit.agg(fun_cagr).reset_index()

In [150]:
ex2_2 = ex2_1.sort_values(['region_1', 'cost_of_unit'], ascending=False).groupby('region_1').head(20)

In [157]:
# 서울특별시 / 부산광역시 / 대전광역시 / 대구광역시 / 광주광역시 / 인천광역시 / 울산광역시 / 세종특별자치시
# 충청북도 / 충청남도 / 전라북도 / 전라남도 / 경상북도 / 경상남도
# 경기도 / 강원도 / 제주특별자치도

# Input region you interested in
sido = ['서울특별시','경기도']

ex2_3 = ex2_2.loc[ex2_2['region_1'].isin(sido)]

In [176]:
ex2_3
# fig2_1 = sns.FacetGrid(ex2_3, row='region_1')
# fig2_1 = fig2_1.map(plt.barh, 'apt_name','cost_of_unit')




,si_gun_gu,region_1,region_2,region_3,apt_name,cost_of_unit
21804,서울특별시 성북구 장위동,서울특별시,성북구,장위동,코코아파트,1.290749
17701,서울특별시 강남구 삼성동,서울특별시,강남구,삼성동,동구싼타빌,1.103213
20947,서울특별시 서대문구 북아현동,서울특별시,서대문구,북아현동,훼미리(1-70),0.939405
21475,서울특별시 서초구 양재동,서울특별시,서초구,양재동,미영(102),0.933797
22228,서울특별시 양천구 목동,서울특별시,양천구,목동,래디앙아파트,0.886069
18544,서울특별시 강서구 내발산동,서울특별시,강서구,내발산동,현대파크,0.883496
23266,서울특별시 은평구 응암동,서울특별시,은평구,응암동,자선메르시안(400),0.683468
17814,서울특별시 강남구 역삼동,서울특별시,강남구,역삼동,(828-76),0.576987
18176,서울특별시 강동구 성내동,서울특별시,강동구,성내동,새한올림픽,0.550870
23303,서울특별시 은평구 증산동,서울특별시,은평구,증산동,청록,0.550000


## 3. 특정 시군구 아파트 매매 현황

In [33]:
region_1_nm = "경기도"
region_2_nm = "성남중원구"
region_3_nm = ""
ex1 = apt[(apt["region_1"] == region_1_nm) & (apt["region_2"] == region_2_nm)]

### 3.1 읍면동별 아파트 매매 현황

In [34]:
pd.pivot_table(ex1, index='region_3', columns='contract_yymm', values='amt_10000', aggfunc='count')

contract_yymm,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812
region_3,,,,,,,,,,,,
금광동,28.0,21.0,44.0,18.0,15.0,22.0,15.0,57.0,42.0,19.0,10.0,2.0
도촌동,43.0,34.0,35.0,25.0,19.0,21.0,15.0,30.0,16.0,6.0,2.0,1.0
상대원동,28.0,22.0,42.0,29.0,23.0,27.0,28.0,66.0,72.0,20.0,20.0,9.0
성남동,15.0,9.0,29.0,13.0,4.0,9.0,6.0,16.0,19.0,9.0,9.0,3.0
여수동,36.0,15.0,7.0,7.0,7.0,8.0,8.0,25.0,19.0,1.0,NaN,NaN
은행동,23.0,28.0,58.0,30.0,25.0,28.0,21.0,53.0,16.0,17.0,13.0,6.0
중앙동,29.0,19.0,36.0,9.0,11.0,7.0,14.0,26.0,12.0,11.0,4.0,2.0
하대원동,25.0,32.0,53.0,20.0,17.0,18.0,22.0,34.0,33.0,14.0,10.0,7.0


### 3.2 아파트매매 건수 상위10개 단지 현황

In [35]:
fig_1 = pd.pivot_table(ex1,index='apt_name',columns='contract_yymm',values='amt_10000',aggfunc='count',margins=True).reset_index()
fig_1 = fig_1.sort_values(['All'], ascending=[False]).head(n=10).iloc[1:,:]
fig_1

contract_yymm,apt_name,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,All
38,선경2차,17.0,9.0,25.0,14.0,15.0,19.0,15.0,50.0,42.0,8.0,7.0,3.0,224
71,현대,14.0,16.0,41.0,12.0,6.0,10.0,13.0,41.0,11.0,7.0,6.0,1.0,178
59,은행주공,9.0,13.0,26.0,21.0,14.0,20.0,11.0,11.0,9.0,9.0,6.0,3.0,152
52,아튼빌,15.0,17.0,27.0,8.0,5.0,6.0,11.0,8.0,14.0,4.0,7.0,3.0,125
19,래미안금광,10.0,12.0,24.0,11.0,6.0,7.0,4.0,17.0,8.0,5.0,2.0,NaN,106
75,황송마을(삼익금광2차),7.0,2.0,11.0,4.0,6.0,9.0,4.0,27.0,10.0,11.0,6.0,NaN,97
78,휴먼시아섬마을5단지,11.0,15.0,10.0,12.0,5.0,5.0,3.0,7.0,6.0,1.0,1.0,NaN,76
67,중앙동힐스테이트2차,15.0,9.0,10.0,5.0,3.0,4.0,8.0,7.0,6.0,5.0,2.0,NaN,74
45,센트럴타운(3단지),25.0,3.0,3.0,3.0,2.0,NaN,5.0,15.0,11.0,NaN,NaN,NaN,67
